In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
plt.rcParams['font.family'] = 'AppleGothic'

In [41]:
df3 = pd.read_csv('../../117.금융 합성데이터/3.개방데이터/1.데이터/1. 합성데이터/10.기업 CB정보/202110_기업CB.csv')

In [ ]:
col_info = pd.read_excel(
    '../../117.금융 합성데이터/3.개방데이터/1.데이터/1. 합성데이터/093-117_금융 합성데이터_데이터구성상세.xlsx',
    sheet_name='10.기업CB 정보',
    header=None,       # 첫 줄을 컬럼명으로 사용하지 않음
    dtype=str          # 타입 강제 → NaN 방지
)

In [58]:
import pandas as pd
import glob

# 1. 병합 셀 정보 파일 (컬럼 영문명 → 설명 맵핑용)
col_info = pd.read_excel(
    '../../117.금융 합성데이터/3.개방데이터/1.데이터/1. 합성데이터/093-117_금융 합성데이터_데이터구성상세.xlsx',
    sheet_name='10.기업CB 정보',
    header=None,       # 첫 줄을 컬럼명으로 사용하지 않음
    dtype=str          # 타입 강제 → NaN 방지
)
col_info_filled = col_info.fillna(method='ffill')

# 영문 필드명과 설명 추출
field_name_col = col_info_filled.iloc[:, 2]
description_col = col_info_filled.iloc[:, 1]

# 딕셔너리 생성: 영문 컬럼명 → 한글 설명
colinfo_description_map = dict(zip(field_name_col, description_col))

# 2. 업종 코드 맵
업종_코드맵 = {
    'A': '농업, 임업 및 어업',
    'B': '광업',
    'C': '제조업',
    'D': '전기, 가스, 증기 및 공기 조절 공급업',
    'E': '수도, 하수 및 폐기물 처리, 원료 재생업',
    'F': '건설업',
    'G': '도매 및 소매업',
    'H': '운수 및 창고업',
    'I': '숙박 및 음식점업',
    'J': '정보통신업',
    'K': '금융 및 보험업',
    'L': '부동산업',
    'M': '전문, 과학 및 기술 서비스업',
    'N': '사업시설 관리, 사업 지원 및 임대 서비스업',
    'O': '공공 행정, 국방 및 사회보장 행정',
    'P': '교육 서비스업',
    'Q': '보건업 및 사회복지 서비스업',
    'R': '예술, 스포츠 및 여가관련 서비스업',
    'S': '협회 및 단체, 수리 및 기타 개인 서비스업'
}

# 3. 모든 CSV 파일 경로 가져오기
csv_paths = sorted(glob.glob("../../117.금융 합성데이터/3.개방데이터/1.데이터/1. 합성데이터/10.기업 CB정보/*.csv"))  # 12개 CSV 경로 폴더 지정

# 4. 병합된 DataFrame 저장용 리스트
merged_list = []

# 5. 각 CSV에 대해 처리 루프
for path in csv_paths:
    df = pd.read_csv(path)

    # 컬럼명 한글로 변환
    df.rename(columns=colinfo_description_map, inplace=True)

    # 업종(대분류) 파생
    df['업종(대분류)'] = df['업종(중분류)'].str[0].map(업종_코드맵)

    # 부동산업만 필터링
    df = df[df['업종(대분류)'].str.contains('부동산업', na=False)]

    # 중분류 제거
    df.drop(columns='업종(중분류)', inplace=True)

    # 리스트에 추가
    merged_list.append(df)

# 6. 최종 병합
final_df = pd.concat(merged_list, ignore_index=True)


/var/folders/kq/wzvd01qd1ql50sz17pwmt9b00000gn/T/ipykernel_68043/1546769447.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  col_info_filled = col_info.fillna(method='ffill')


In [59]:
final_df.to_csv('final_부동산.csv', index=False)

In [60]:
final_df.to_excel('final_부동산.xlsx', index=False)